In [46]:
import tqdm
import torch
import wandb
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from peft import LoraConfig, get_peft_model, TaskType
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW

In [47]:
wandb.init(project="LoRA MobileBERT Model FineTuning Yo!")

In [48]:
dataStuff = pd.read_csv("../augmentedDataStuff.csv")
dataStuff

,OCR Contents,Category
0,Invoice Number: INV0786Hotel Name: Spice Garde...,Food Category - Restaurant Bill/Receipt
1,Invoice No- DW/1112/851166\n\nInvoice Date : 2...,Travel Category - Flight Ticket
2,Hotel: Accor\r\nCity: Delhi\r\nNumber of Days:...,Miscellaneous Category - Miscellaneous Bills
3,FOOD INN FINE DINE\nProprietor\nPER =NNIA REST...,Food Category - Restaurant Bill/Receipt
4,= Confirmed Booking Id: NF20115276689422\n\nSU...,Travel Category - Flight Ticket
...,...,...
5161,Indiandit\n\nSERIAL SNA AUTONOBI\n\nr .A.B.Roa...,Fuel Category - Fuel Bill/Receipt
5162,Base Fare\nDistance\nTime\n\nSubtotal\n\nCHARG...,Travel Category - Cab Bill/Receipt
5163,"Dear sanjay kumar,\n\nCongratdatons! Thark you...",Travel Category - Train Ticket
5164,Invoice Number: INV0780Hotel Name: Taste of It...,Food Category - Restaurant Bill/Receipt


In [49]:
clsStuff = list(set(dataStuff["Category"]))
clsStuff

['Room Stay Category - Stay Bill/Receipt',
 'Travel Category - Train Ticket',
 'Food Category - Restaurant Bill/Receipt',
 'Travel Category - Cab Bill/Receipt',
 'Travel Category - Flight Ticket',
 'Electronics Category - Electronic Gadgets Bill/Receipt',
 'Fuel Category - Fuel Bill/Receipt',
 'Travel Category - Bus Ticket',
 'Miscellaneous Category - Miscellaneous Bills']

In [50]:
labelEncoder = LabelEncoder()
labelEncoder.fit(clsStuff)
clsDict = dict(zip(clsStuff, labelEncoder.transform(clsStuff)))
clsDict

{'Room Stay Category - Stay Bill/Receipt': 4,
 'Travel Category - Train Ticket': 8,
 'Food Category - Restaurant Bill/Receipt': 1,
 'Travel Category - Cab Bill/Receipt': 6,
 'Travel Category - Flight Ticket': 7,
 'Electronics Category - Electronic Gadgets Bill/Receipt': 0,
 'Fuel Category - Fuel Bill/Receipt': 2,
 'Travel Category - Bus Ticket': 5,
 'Miscellaneous Category - Miscellaneous Bills': 3}

In [51]:
numLabels = len(clsStuff)
batchSize = 16
learningRate = 2e-5
epochs = 250

In [52]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [60]:
model = AutoModelForSequenceClassification.from_pretrained("google/mobilebert-uncased", num_labels=numLabels).to(device)
tokenizer = AutoTokenizer.from_pretrained("google/mobilebert-uncased")

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [61]:
model.config

MobileBertConfig {
  "_name_or_path": "google/mobilebert-uncased",
  "architectures": [
    "MobileBertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_activation": false,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 512,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "intra_bottleneck_size": 128,
  "key_query_shared_bottleneck": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "mobilebert",
  "normalization_type": "no_norm",
  "num_attention_heads": 4,
  "num_f

In [62]:
loraConfig = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [63]:
model = get_peft_model(model, loraConfig)

In [65]:
wandb.watch(model)
wandb.config["learning_rate"] = learningRate
wandb.config["batch_size"] = batchSize

In [66]:
trainData, valData = train_test_split(dataStuff, test_size=0.2, random_state=42)
trainData['OCR Contents']

4711    Hotel: Hilton Worldwide\r\nCity: Amritsar\r\nN...
4184    goibibo sao\n\nFlight Ticket - Kolkata to Agar...
1670    Category: Utilities\r\nItem: Internet Bill\r\n...
99      Invoice Number: INV0810Hotel Name: The Hungry ...
3592    Invoice Number: INV0953Hotel Name: Gourmet Pal...
                              ...                        
4426    Invoice Number: INV0558Hotel Name: The Hungry ...
466     ticket ric 0: r07s220168 Gcleartrip\n\nBangalo...
3092    Category: Shopping\r\nItem: Accessories\r\nAmo...
3772    " ‘\n\nSREP iy\n\nCHOKHI DHANI PALACE HOTEL\n\...
860     Category: Healthcare\r\nItem: Medicines\r\nAmo...
Name: OCR Contents, Length: 4132, dtype: object

In [67]:
def prepData(data):
  train_inputs, train_masks, train_labels = [], [], []
  for item, label in zip(data["OCR Contents"], data["Category"]):
    text_content = item
    encoded_data = tokenizer(text_content, padding="max_length", truncation=True, max_length=512)
    train_inputs.append(encoded_data["input_ids"])
    train_masks.append(encoded_data["attention_mask"])
    train_labels.append(label)
  return train_inputs, train_masks, train_labels

In [68]:
def labels2Encoding(labelStuff):
    encodedLabels = labelEncoder.transform(labelStuff)
    return encodedLabels

In [69]:
trainInputs, trainMasks, trainLabels = prepData(trainData)
valInputs, valMasks, valLabels = prepData(valData)

In [70]:
trainLabels

['Miscellaneous Category - Miscellaneous Bills',
 'Travel Category - Bus Ticket',
 'Miscellaneous Category - Miscellaneous Bills',
 'Food Category - Restaurant Bill/Receipt',
 'Food Category - Restaurant Bill/Receipt',
 'Travel Category - Train Ticket',
 'Miscellaneous Category - Miscellaneous Bills',
 'Food Category - Restaurant Bill/Receipt',
 'Room Stay Category - Stay Bill/Receipt',
 'Miscellaneous Category - Miscellaneous Bills',
 'Food Category - Restaurant Bill/Receipt',
 'Miscellaneous Category - Miscellaneous Bills',
 'Miscellaneous Category - Miscellaneous Bills',
 'Miscellaneous Category - Miscellaneous Bills',
 'Food Category - Restaurant Bill/Receipt',
 'Miscellaneous Category - Miscellaneous Bills',
 'Travel Category - Bus Ticket',
 'Miscellaneous Category - Miscellaneous Bills',
 'Travel Category - Flight Ticket',
 'Food Category - Restaurant Bill/Receipt',
 'Food Category - Restaurant Bill/Receipt',
 'Travel Category - Train Ticket',
 'Travel Category - Bus Ticket',
 'T

In [71]:
trainLabels = labels2Encoding(trainLabels)
valLabels = labels2Encoding(valLabels)
trainLabels

array([3, 5, 3, ..., 3, 1, 3])

In [72]:
trainInputs = torch.tensor(trainInputs, dtype=torch.uint8).to(device)
trainMasks = torch.tensor(trainMasks, dtype=torch.uint8).to(device)
trainLabels = torch.tensor(trainLabels, dtype=torch.uint8).to(device)
valInputs = torch.tensor(valInputs, dtype=torch.uint8).to(device)
valMasks = torch.tensor(valMasks, dtype=torch.uint8).to(device)
valLabels = torch.tensor(valLabels, dtype=torch.uint8).to(device)

In [73]:
print(trainMasks.max(), trainInputs.min())

tensor(1, device='cuda:0', dtype=torch.uint8) tensor(0, device='cuda:0', dtype=torch.uint8)


In [74]:
trainDataset = TensorDataset(trainInputs, trainMasks, trainLabels)
trainDataloader = DataLoader(trainDataset, batch_size=batchSize, shuffle=True)

In [75]:
valDataset = TensorDataset(valInputs, valMasks, valLabels)
valDataloader = DataLoader(valDataset, batch_size=batchSize)

In [76]:
optimizer = AdamW(model.parameters(), lr=learningRate)
criterion = torch.nn.CrossEntropyLoss()

c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [77]:
def train(model, optimizer, criterion, trainDataloader):
  model.train()
  total_loss = 0
  correct = 0
  for batch_idx, (input_ids, attention_mask, labels) in enumerate(trainDataloader):
    optimizer.zero_grad()
    input_ids = input_ids.long().to(device)
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    loss = criterion(outputs.logits, labels)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    predictions = torch.argmax(outputs.logits, dim=1)
    correct += (predictions == labels).sum().item()
  return total_loss / len(trainDataloader), correct / len(trainDataset)


In [78]:
def validate(model, criterion, valDataloader):
  model.eval()
  total_loss = 0
  correct = 0
  all_preds, all_labels = [], []
  with torch.no_grad():
    for batch_idx, (input_ids, attention_mask, labels) in enumerate(valDataloader):
      input_ids = input_ids.long().to(device)
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      loss = criterion(outputs.logits, labels)
      total_loss += loss.item()
      predictions = torch.argmax(outputs.logits, dim=1)
      correct += (predictions == labels).sum().item()
      all_preds.extend(predictions.cpu().numpy())
      all_labels.extend(labels.cpu().numpy())
  accuracy = accuracy_score(all_labels, all_preds)
  precision = precision_score(all_labels, all_preds, average="weighted")
  recall = recall_score(all_labels, all_preds, average="weighted")
  return total_loss / len(valDataloader), accuracy, precision, recall

In [79]:
for epoch in tqdm.trange(epochs, colour = "red", desc = "Epoch(s)"):
  train_loss, train_acc = train(model, optimizer, criterion, trainDataloader)
  val_loss, val_acc, val_precision, val_recall = validate(model, criterion, valDataloader)
  dataDict = {
    "Epoch": epoch+1,
    "Train Loss": train_loss,
    "Train Accuracy": train_acc,
    "Val Loss": val_loss,
    "Val Accuracy": val_acc,
    "Val Precision": val_precision,
    "Val Recall": val_recall
    }
  print(dataDict)
  wandb.log(dataDict)
  model.save_pretrained("tunedModels/trainedMobileBERTLoRA - Epoch {}".format(epoch))

Epoch(s):   0%|          | 0/250 [00:00<?, ?it/s]


TypeError: forward() got an unexpected keyword argument 'decoder_input_ids'

In [ ]:
model.save_pretrained("trainedMobileBERT")